In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# df_mfe = pd.read_csv("/Users/bryan/Studio/tools/mix_10.csv")
# df_mfe = pd.read_csv('/Users/bryan/Studio/seq/data/samples/21.csv')
# df_mfe = pd.read_excel('/Users/bryan/Documents/SeqDataSets/200313/200313s10_100.xls')
# df_mfe = pd.read_csv('/Users/bryan/Studio/seq2/compounds.csv')
# df_mfe = pd.read_csv('/Users/bryan/Documents/SeqDataSets/signal_to_noise_20/SLZ_200313_10_0610_20_k.csv')
# df_mfe = pd.read_csv('/Users/bryan/Documents/SeqDataSets/signal_to_noise_5/SLZ_200313_10_0606_z.csv')
df_mfe = pd.read_excel('/Users/bryan/Documents/SeqDataSets/signal_to_noise_5/SLZ_200313_s09_0619_qs.xls')
df_mfe.shape

In [ ]:
plt.figure(figsize=(12, 9))
plt.scatter(df_mfe['Mass'], df_mfe['RT'])
plt.xlabel('Mass')
plt.ylabel('RT')

In [ ]:
TAG = 826.3184
# TAG = 694.2397
# endmass_3p = 6781.0487 #6781.0763 #19
# endmass_3p = 7079.0771 #7079.0861 #20
# endmass_3p = 7522.1059 #7522.1079 #21
# endmass_3p = 7088.0804 #7088.0775 #201
# endmass_3p = 6985.0463 #6985.0417 #202
# endmass_3p = 7073.0771 #7073.0735 #203
# endmass_3p = 6954.9782 #6954.9831 #204
# endmass_3p = 7110.0904 #7110.0883 #205
# endmass_3p = 7151.113 #206
# endmass_3p = 7193.0523 #207

endmass_3p = 8832.4031 #8832.3519 #s09
# endmass_3p = 9501.5099 #9501.4446 #s10
# endmass_3p = 10190.5566 #s12
# endmass_3p = 7234.1721 #s13
sum_value = round(endmass_3p + 18.0106, 1)

endmass_5p = sum_value - 80 - TAG
masses = df_mfe.Mass
BIAS = 0.1
end5p_idxs = masses[(masses>=endmass_5p-BIAS) & (masses<=endmass_5p+BIAS)].index
end5p_df = df_mfe.loc[end5p_idxs]
print("5' ladder end mass {}, found \n{}".format(endmass_5p, end5p_df[['Mass', 'RT', 'Vol']]))

masses = df_mfe.Mass
end3p_idxs = masses[(masses>=endmass_3p-BIAS) & (masses<=endmass_3p+BIAS)].index
end3p_df = df_mfe.loc[end3p_idxs]
print("3' ladder end mass {}, found \n{}".format(endmass_3p, end3p_df[['Mass', 'RT', 'Vol']]))

In [ ]:
end5p_df = end5p_df.drop(1124)
end3p_df = end3p_df.drop(256)

In [ ]:
end5p_idx = end5p_df['Vol'].idxmax()
df5p_endpoint = end5p_df.loc[end5p_idx]
df5p_endpoint[['Mass', 'RT', 'Vol']]

In [ ]:
PPM = 10
base_masses = [305.0413, 329.0525, 345.0474, 306.0253, 359.0631, 308.041, 319.0569, 320.041, 373.0787, 343.0682, 570.1475, 212.0086, 557.2251, 688.1156, 625.0844]

df = df_mfe.sort_index()
df5p = start5p_df = pd.DataFrame([[18.0106, 0.01, 10000],
                          [347.0631, 0.01, 10000],
                          [323.0519, 0.01, 10000], 
                          [363.058, 0.01, 10000], 
                          [324.0359, 0.01, 10000]], columns=['Mass', 'RT', 'Vol']).reindex()
df3p = start3p_df = pd.DataFrame([[18.0106, 0.01, 10000], [TAG, 6.66, 20000]], columns=['Mass', 'RT', 'Vol']).reindex()



In [ ]:
df3p = start3p_df = pd.DataFrame([[TAG, 6.66, 20000]], columns=['Mass', 'RT', 'Vol']).reindex()

In [ ]:

df5p = start5p_df = pd.DataFrame([
                          [347.0631, 0.01, 10000],
                          [323.0519, 0.01, 10000], 
                          [363.058, 0.01, 10000], 
                          [324.0359, 0.01, 10000]], columns=['Mass', 'RT', 'Vol']).reindex()

In [ ]:
# x = np.array([668.0955, 973.1367, 5775.7534, 6024.8510])
# y = np.array([0.7340, 0.8460, 7.781, 7.596])
x_ends = np.array([668.0955, 6024.8510])
y_ends = np.array([0.7340, 7.596])
z = np.polyfit(x_ends, y_ends, 1)
def f(x):
    return z[0]*x + z[1]
#     return z[0]*x*x + z[1]*x + z[2]
# x = common_df5p.Mass.sort_values()
plt.scatter(df_mfe['Mass'], df_mfe['RT'])
plt.plot(x_ends, f(x_ends))
plt.plot(x_ends, f(x_ends)+1.5)
plt.plot(x_ends, f(x_ends)-0.5)
print(z)

In [ ]:
def basecalling_minus(df_x, df_y, rt_strict=False, untagged=False):
    print("basecalling_plus params {} {}".format(df_x, df_y.shape))
    df_total = df_x.copy()
    if untagged:
        df_x.Mass = df_x.Mass + 80
    while True:
        idxs = list()
        mass_x = np.array(df_x.Mass)
        mass_y = np.array(df_y.Mass)
        rt_x = np.array(df_x.RT)
        rt_y = np.array(df_y.RT)
        for base_mass in base_masses:
#             ppm_matrix = np.abs((np.abs(mass_x[:, np.newaxis] - mass_y) - base_mass) * 1E6 / (mass_y + base_mass))
            ppm_matrix = np.abs((mass_x[:, np.newaxis] - mass_y - base_mass) * 1E6 /  (mass_y + base_mass))
#             ppm_matrix = np.abs((mass_x[:, np.newaxis] - mass_y - base_mass) * 1E6 /  mass_y)
#             ppm_matrix = np.abs((-mass_x[:, np.newaxis] + mass_y - base_mass) * 1E6 / base_mass)
            within_ppm10 = ppm_matrix < 10
            ppm_df = pd.DataFrame(ppm_matrix)
#             base_connects = ppm_df[ppm_df < 10]

            rt_matrix = rt_x[:, np.newaxis] - rt_y
            if rt_strict:
                diff_rt_low = 0
            else:
                diff_rt_low = -0.5
            is_rt_asc = (rt_matrix > diff_rt_low) & (rt_matrix < 1.8)
            mask = within_ppm10# & is_rt_asc
            
            mask_df = pd.DataFrame(mask)
            base_connects = ppm_df[mask_df]
            
            idx_pairs = list(base_connects[base_connects.notnull()].stack().index)
            df5_idxs = [pair[1] for pair in idx_pairs]
            df5_idxs = list(set(df5_idxs))
            if df5_idxs:
                idxs.extend(df5_idxs)

        if not idxs:
            break
        idxs = list(set(idxs))
        if idxs:
            print("number of indexes {}".format(len(idxs)))
        moved = df_y.iloc[idxs]
        mass_x = np.append(mass_x, np.array(moved.Mass))
#         if untagged:
#             df_x = moved[(moved.RT >= f(moved.Mass)-0.5) & (moved.RT <= f(moved.Mass)+1.5)]
#         else:
        df_x = moved
        df_total = df_total.append(df_x)
        df_y = df_y.drop(moved.index)
        mass_y = np.delete(mass_y, idxs)
    
    print(df_total.shape)
    return df_total, df_y

In [ ]:
def basecalling_plus(df_x, df_y, rt_strict=False):
    print("basecalling_plus params {} {} {}".format(df_x.shape, df_y.shape, rt_strict))
    df_total = df_x.copy()
    while True:
        idxs = list()
        mass_x = np.array(df_x.Mass)
        mass_y = np.array(df_y.Mass)
        rt_x = np.array(df_x.RT)
        rt_y = np.array(df_y.RT)
        for base_mass in base_masses:
            ppm_matrix = np.abs((-mass_x[:, np.newaxis] + mass_y - base_mass) * 1E6 /  (mass_y + base_mass))
            within_ppm10 = ppm_matrix < 10
            ppm_df = pd.DataFrame(ppm_matrix)
#             base_connects = ppm_df[ppm_df < 10]
            
            rt_matrix = -rt_x[:, np.newaxis] + rt_y
            if rt_strict:
                diff_rt_low = 0
            else:
                diff_rt_low = -0.5
            is_rt_asc = (rt_matrix > diff_rt_low) & (rt_matrix < 1.5)
            mask = within_ppm10 #& is_rt_asc
            
            mask_df = pd.DataFrame(mask)
            base_connects = ppm_df[mask_df]
            
            idx_pairs = list(base_connects[base_connects.notnull()].stack().index)
            df5_idxs = [pair[1] for pair in idx_pairs]
            df5_idxs = list(set(df5_idxs))
            if df5_idxs:
                idxs.extend(df5_idxs)

        if not idxs:
            break
        idxs = list(set(idxs))
        if idxs:
            print("number of indexes {}".format(len(idxs)))
        moved = df_y.iloc[idxs]
        mass_x = np.append(mass_x, np.array(moved.Mass))
        df_total = df_total.append(moved)
        df_x = moved
        df_y = df_y.drop(moved.index)
        mass_y = np.delete(mass_y, idxs)
    
    print(df_total.shape)
    return df_total, df_y

In [ ]:
df3p, _ = basecalling_plus(df3p, df)

In [ ]:
end3p_df

In [ ]:
df3p_minus, _ = basecalling_minus(end3p_df, df)

In [ ]:
df5p, _ = basecalling_plus(df5p, df, rt_strict=True)

In [ ]:
plt.scatter(df5p.Mass, df5p.RT)

In [ ]:
df5p_minus, _ = basecalling_minus(end5p_df, df, untagged=True)

In [ ]:
df3p.shape, df5p.shape, df.shape

In [ ]:
df3p_minus.shape, df5p_minus.shape, df.shape

In [ ]:
df5p = df5p.append(end5p_df.loc[end5p_idx])
df3p = df3p.append(end3p_df)
df5p_minus = df5p_minus.append(end5p_df.loc[end5p_idx])
df3p_minus = df3p_minus.append(end3p_df)

In [ ]:
plt.figure(figsize=(12, 9))
# plt.scatter(df5p.Mass, df5p.RT)
plt.scatter(df3p.Mass, df3p.RT)
# plt.scatter(end5p_df.loc[end5p_idx].Mass, end5p_df.loc[end5p_idx].RT, marker='X', color='r')
plt.scatter(end3p_df.Mass, end3p_df.RT, marker='X', color='r')
# plt.scatter(start5p_df.Mass, start5p_df.RT, marker='X', color='r')
plt.scatter(start3p_df.Mass, start3p_df.RT, marker='X', color='r')
plt.xlabel('Mass')
plt.ylabel('RT')

In [ ]:
plt.figure(figsize=(12, 9))
# plt.scatter(df5p_minus.Mass, df5p_minus.RT)
plt.scatter(df3p_minus.Mass, df3p_minus.RT)
# plt.scatter(end5p_df.loc[end5p_idx].Mass, end5p_df.loc[end5p_idx].RT, marker='X', color='r')
plt.scatter(end3p_df.Mass, end3p_df.RT, marker='X', color='r')
# plt.scatter(start5p_df.Mass, start5p_df.RT, marker='X', color='r')

In [ ]:
df3p.to_excel('data/3p.xlsx')
df5p.to_excel('data/5p.xlsx')
df_merge = pd.concat([df3p, df5p])
df_merge.to_csv('data/merge.csv')

In [ ]:
df3p_minus.to_excel('data/3p.xlsx')
df5p_minus.to_excel('data/5p.xlsx')
df_merge = pd.concat([df3p_minus, df5p_minus])
df_merge.to_csv('data/merge.csv')

In [ ]:
df5p[df5p.Mass > 5000][['Mass', 'RT', 'Vol']]

In [ ]:
df5p_minus[(df5p_minus.Mass < 975) & (df5p_minus.Mass > 972)][['Mass', 'RT', 'Vol']].mean()

In [ ]:
plt.scatter(df5p_minus.Mass, df5p_minus.RT)

In [ ]:
common_idxs_5p = df5p_minus.index & df5p.index
common_idxs_3p = df3p_minus.index & df3p.index

In [ ]:
common_idxs_5p.shape, common_idxs_3p.shape

In [ ]:
common_df3p = df_mfe.loc[common_idxs_3p]
common_df5p = df_mfe.loc[common_idxs_5p]
plt.figure(figsize=(12, 9))
# plt.scatter(common_df5p.Mass, common_df5p.RT)
plt.scatter(common_df3p.Mass, common_df3p.RT)
# plt.scatter(end5p_df.loc[end5p_idx].Mass, end5p_df.loc[end5p_idx].RT, marker='X', color='r')
plt.scatter(end3p_df.Mass, end3p_df.RT, marker='X', color='r')
# plt.scatter(start5p_df.Mass, start5p_df.RT, marker='X', color='r')

x = np.array([668.0955, 973.1367, 5775.7534, 6024.8510])
y = np.array([0.7340, 0.8460, 7.781, 7.596])
z = np.polyfit(x, y, 1)
def f(x):
    return z[0]*x + z[1]
#     return z[0]*x*x + z[1]*x + z[2]
# x = common_df5p.Mass.sort_values()
# plt.plot(x, f(x))
# plt.plot(x, f(x)+1.5)
# plt.plot(x, f(x)-0.5)
print(z)
print(common_df3p.shape, common_df5p.shape)

In [ ]:
common_df5p.shape

In [ ]:
df5p_tmp = common_df5p[(common_df5p.RT >= f(common_df5p.Mass)-0.5) & (common_df5p.RT <= f(common_df5p.Mass)+1.5)]
plt.figure(figsize=(16, 12))
plt.scatter(df5p_tmp['Mass'], df5p_tmp['RT'])
plt.scatter(end5p_df['Mass'], end5p_df['RT'], marker='X')
print(df5p_tmp.shape)

In [ ]:
plt.scatter(df3p_minus.Mass, df3p_minus.RT)
# plt.scatter(end5p_df.loc[end5p_idx].Mass, end5p_df.loc[end5p_idx].RT, marker='X', color='r')
plt.scatter(end3p_df.Mass, end3p_df.RT, marker='X', color='r')
plt.scatter(start5p_df.Mass, start5p_df.RT, marker='X', color='r')

In [ ]:
common_df3p = common_df3p[['Cpd', 'Mass', 'RT', 'Vol', 'Width', 'Quality Score']]
common_df5p = common_df5p[['Cpd', 'Mass', 'RT', 'Vol', 'Width', 'Quality Score']]
common_df3p.to_excel('data/3p.xlsx')
common_df5p.to_excel('data/5p.xlsx')
df_merge = pd.concat([df3p_minus, df5p_minus])
df_merge.to_csv('data/merge.csv')

In [ ]:
common_df5p.shape

In [ ]:
seqs_df = pd.read_csv('/Users/bryan/Studio/seq2/seqs.csv')

In [ ]:
seqs_df = seqs_df[seqs_df.RT < 10]
plt.scatter(seqs_df.Mass, seqs_df.RT)